In [4]:
from ultralytics import YOLO
from ultralytics.data.augment import Albumentations
from ultralytics.utils import LOGGER, colorstr
import os 
import cv2

In [5]:
import os
import cv2
import albumentations as A

# === YOUR PATHS ===
image_dir = r"C:\Users\Welcome\Desktop\dataset\images"
label_dir = r"C:\Users\Welcome\Desktop\dataset\labels"
output_img_dir = r"C:\Users\Welcome\Desktop\dataset\augmented_images"
output_lbl_dir = r"C:\Users\Welcome\Desktop\dataset\augmented_labels"

# Create output folders
os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_lbl_dir, exist_ok=True)

# === 6 DISTINCT AUGMENTATIONS ===
augmentations = [
    A.Compose([A.HorizontalFlip(p=1)], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"])),
    A.Compose([A.RandomBrightnessContrast(p=1)], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"])),
    A.Compose([A.Rotate(limit=30, p=1)], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"])),
    A.Compose([A.ISONoise(p=1)], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"])),
    A.Compose([A.RGBShift(p=1)], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"])),
    A.Compose([A.ElasticTransform(alpha=1, sigma=50, alpha_affine=30, p=1)], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]))
]

# === PROCESS EACH IMAGE ===
for filename in os.listdir(image_dir):
    if not filename.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    base = os.path.splitext(filename)[0]
    img_path = os.path.join(image_dir, filename)
    label_path = os.path.join(label_dir, base + ".txt")

    if not os.path.exists(label_path):
        print(f"[!] Skipping {filename}, no matching label found.")
        continue

    # Load image
    image = cv2.imread(img_path)
    h, w = image.shape[:2]

    # Load YOLO bboxes
    bboxes = []
    class_labels = []
    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            cls, cx, cy, bw, bh = map(float, parts)
            bboxes.append([cx, cy, bw, bh])
            class_labels.append(int(cls))

    if not bboxes:
        continue

    # Apply all 6 augmentations
    for i, aug in enumerate(augmentations):
        try:
            result = aug(image=image, bboxes=bboxes, class_labels=class_labels)
        except Exception as e:
            print(f"[!] Error augmenting {filename} (aug {i}): {e}")
            continue

        aug_img = result["image"]
        aug_boxes = result["bboxes"]

        # Save augmented image
        aug_img_name = f"{base}_aug_{i}.jpg"
        aug_img_path = os.path.join(output_img_dir, aug_img_name)
        cv2.imwrite(aug_img_path, aug_img)

        # Save updated YOLO label
        aug_lbl_name = f"{base}_aug_{i}.txt"
        aug_lbl_path = os.path.join(output_lbl_dir, aug_lbl_name)
        with open(aug_lbl_path, "w") as f:
            for cls, box in zip(class_labels, aug_boxes):
                f.write(f"{cls} {' '.join(map(str, box))}\n")

        print(f"[✓] Saved: {aug_img_name}")


c:\Users\Welcome\phases_detection\model_testing\Lib\site-packages\albumentations\core\composition.py:250: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()
C:\Users\Welcome\AppData\Local\Temp\ipykernel_21824\3816450154.py:22: UserWarning: Argument(s) 'alpha_affine' are not valid for transform ElasticTransform
  A.Compose([A.ElasticTransform(alpha=1, sigma=50, alpha_affine=30, p=1)], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]))


[✓] Saved: image_0000_0-11-_jpg.rf.be356433ff780c2e609cb9690bd86b26_aug_0.jpg
[✓] Saved: image_0000_0-11-_jpg.rf.be356433ff780c2e609cb9690bd86b26_aug_1.jpg
[✓] Saved: image_0000_0-11-_jpg.rf.be356433ff780c2e609cb9690bd86b26_aug_2.jpg
[✓] Saved: image_0000_0-11-_jpg.rf.be356433ff780c2e609cb9690bd86b26_aug_3.jpg
[✓] Saved: image_0000_0-11-_jpg.rf.be356433ff780c2e609cb9690bd86b26_aug_4.jpg
[✓] Saved: image_0000_0-11-_jpg.rf.be356433ff780c2e609cb9690bd86b26_aug_5.jpg
[✓] Saved: image_0000_0-13-_jpg.rf.dbc36a0dcb2fa39d53792a696ccfb8d9_aug_0.jpg
[✓] Saved: image_0000_0-13-_jpg.rf.dbc36a0dcb2fa39d53792a696ccfb8d9_aug_1.jpg
[✓] Saved: image_0000_0-13-_jpg.rf.dbc36a0dcb2fa39d53792a696ccfb8d9_aug_2.jpg
[✓] Saved: image_0000_0-13-_jpg.rf.dbc36a0dcb2fa39d53792a696ccfb8d9_aug_3.jpg
[✓] Saved: image_0000_0-13-_jpg.rf.dbc36a0dcb2fa39d53792a696ccfb8d9_aug_4.jpg
[✓] Saved: image_0000_0-13-_jpg.rf.dbc36a0dcb2fa39d53792a696ccfb8d9_aug_5.jpg
[✓] Saved: image_0000_0-14-_jpg.rf.9ef8024b1d69355d6032dcd7986f4

In [8]:
import os
import cv2
import random

# === PATHS ===
image_dir = r"C:\Users\Welcome\Desktop\dataset\augmented_images"
label_dir = r"C:\Users\Welcome\Desktop\dataset\augmented_labels"
output_dir = r"C:\Users\Welcome\Desktop\dataset\augmented_preview"
os.makedirs(output_dir, exist_ok=True)

# === FILTER ONE IMAGE FOR EACH AUG INDEX (aug_0 to aug_5) ===
augmented_images = {}
for file in os.listdir(image_dir):
    if file.endswith(('.jpg', '.png', '.jpeg')) and "_aug_" in file:
        try:
            aug_index = int(file.split("_aug_")[1].split(".")[0])
            if aug_index not in augmented_images:
                augmented_images[aug_index] = file
            if len(augmented_images) >= 6:
                break
        except:
            continue

# === DRAW BOXES AND SAVE ===
for aug_index, file in augmented_images.items():
    img_path = os.path.join(image_dir, file)
    label_path = os.path.join(label_dir, os.path.splitext(file)[0] + '.txt')

    img = cv2.imread(img_path)
    if img is None:
        print(f"[!] Could not load image: {file}")
        continue

    h, w = img.shape[:2]

    if not os.path.exists(label_path):
        print(f"[!] No label found for: {file}")
        continue

    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            cls, cx, cy, bw, bh = map(float, parts)
            x1 = int((cx - bw / 2) * w)
            y1 = int((cy - bh / 2) * h)
            x2 = int((cx + bw / 2) * w)
            y2 = int((cy + bh / 2) * h)
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, f"Class {int(cls)}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Save visualized image
    save_path = os.path.join(output_dir, f"preview_{file}")
    cv2.imwrite(save_path, img)
    print(f"[✓] Saved preview: {save_path}")


[✓] Saved preview: C:\Users\Welcome\Desktop\dataset\augmented_preview\preview_image_0000_0-11-_jpg.rf.be356433ff780c2e609cb9690bd86b26_aug_0.jpg
[✓] Saved preview: C:\Users\Welcome\Desktop\dataset\augmented_preview\preview_image_0000_0-11-_jpg.rf.be356433ff780c2e609cb9690bd86b26_aug_1.jpg
[✓] Saved preview: C:\Users\Welcome\Desktop\dataset\augmented_preview\preview_image_0000_0-11-_jpg.rf.be356433ff780c2e609cb9690bd86b26_aug_2.jpg
[✓] Saved preview: C:\Users\Welcome\Desktop\dataset\augmented_preview\preview_image_0000_0-11-_jpg.rf.be356433ff780c2e609cb9690bd86b26_aug_3.jpg
[✓] Saved preview: C:\Users\Welcome\Desktop\dataset\augmented_preview\preview_image_0000_0-11-_jpg.rf.be356433ff780c2e609cb9690bd86b26_aug_4.jpg
[✓] Saved preview: C:\Users\Welcome\Desktop\dataset\augmented_preview\preview_image_0000_0-11-_jpg.rf.be356433ff780c2e609cb9690bd86b26_aug_5.jpg
